In [0]:
import sys
sys.path.append("/Workspace/SPTrans_v2.0")

from datetime import datetime
from core.config_loader import load_config
from core.sptrans_client import SPTransClient

config = load_config("/Workspace/SPTrans_v2.0/config.json")

sp = SPTransClient(
    base_url=config["sptrans"]["base_url"],
    token=config["sptrans"]["token"],
    proxies=config["sptrans"].get("proxy")
)

def coletar_posicoes():
    if not sp.authenticate():
        print("❌ Falha ao autenticar")
        return
    
    data = sp.get("Posicao")
    if not data:
        print("❌ Erro na API /Posicao")
        return
    
    hr = data.get("hr", "N/D")
    registros = []

    for linha in data.get("l", []):
        for v in linha.get("vs", []):
            registros.append({
                "prefixo": v.get("p"),
                "acessivel": v.get("a"),
                "ta_utc": v.get("ta"),
                "lat": v.get("py"),
                "lng": v.get("px"),
                "cod_linha": linha.get("cl"),
                "letreiro": linha.get("c"),
                "sentido": linha.get("sl"),
                "destino": linha.get("lt0"),
                "origem": linha.get("lt1"),
                "hr_api": hr,
                "ts_coleta": datetime.now()
            })

    if not registros:
        print("⚠ Nenhum veículo localizado")
        return

    df = spark.createDataFrame(registros)

    df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.sptrans.bronze_posicao")

    print(f"✅ {len(registros)} veículos salvos em bronze_posicao")

coletar_posicoes()

✅ Autenticado na API SPTrans
✅ 11319 veículos salvos em bronze_posicao


In [0]:
%sql
SELECT COUNT(*) AS total_registros
FROM workspace.sptrans.bronze_posicao;

total_registros
4730
